In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
input_path = "C:/Users/AS-GP/Desktop/Resnet50/lisat_gaze_data_v1/"

In [15]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(), #for parallel computations
        normalize
    ]),
}

#train_subset_size = int(0.4 * len(datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/train', transform=data_transforms['train']))) #testing hashelha b3den
#val_subset_size = int(0.4 * len(datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/val', transform=data_transforms['validation'])))
image_datasets = {
    'train':
    datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/train', data_transforms['train']),
    'validation':
    datasets.ImageFolder(input_path + 'lisat_gaze_data_v1/val', data_transforms['validation'])
}


In [5]:
model =torchvision.models.mobilenet_v2(pretrained=True).to(device)
model

C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AS-GP\AppData\Local\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [7]:
model =torchvision.models.mobilenet_v2(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1280, out_features=8, bias=True),
            nn.Softmax(dim=1))

model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [11]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.classifier.parameters())
# Define the grid of hyperparameters
batch_sizes = [16, 32, 64]
learning_rates = [0.0001, 0.001, 0.1]
weight_decays = [0.0001, 0.001, 0.01]



In [12]:
num_epochs = 5

In [17]:
def train_and_evaluate(batch_size, learning_rate , weight_decay , train_loader, test_loader):
    optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate, weight_decay=weight_decay)
    for epoch in range(num_epochs):
        best_val_accuracy =0;
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train().to(device) #added to device
                data_loader = train_loader
            else:
                model.eval()
                data_loader = test_loader
            running_loss = 0.0
            running_corrects = 0
            

            
            for data, target in data_loader:
                data = data.to(device)
                target = target.to(device)

                output = model(data)
                loss = criterion(output, target)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(output, 1)
                running_loss += loss.item() * data.size(0)
                running_corrects += torch.sum(preds == target.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))

            if(phase == 'validation' and epoch_acc > best_val_accuracy):
                    best_val_accuracy = epoch_acc
            
    return best_val_accuracy
                
        


In [18]:
# Perform the grid search
best_parameters = ()
best_accuracy = 0.0
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        for weight_decay in weight_decays:
            print(f"Batch size: {batch_size}, Learning rate: {learning_rate}, Weight decay: {weight_decay}")
            train_loader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True)
            test_loader = torch.utils.data.DataLoader(image_datasets['validation'], batch_size=batch_size, shuffle=False)
            accuracy = train_and_evaluate(batch_size, learning_rate, weight_decay , train_loader , test_loader)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters = (batch_size, learning_rate, weight_decay)
                print(f"Current Best accuracy: {best_accuracy}")
                print(f"Current Best parameters: {best_parameters}")

print(f" Final Best parameters: {best_parameters}")
print(f"Final Best accuracy: {best_accuracy}")      

Batch size: 16, Learning rate: 0.0001, Weight decay: 0.0001
Epoch 1/5
----------
train loss: 1.9418, acc: 0.3574
validation loss: 1.8664, acc: 0.4664
Epoch 2/5
----------
train loss: 1.8285, acc: 0.5205
validation loss: 1.7791, acc: 0.5431
Epoch 3/5
----------
train loss: 1.7838, acc: 0.5549
validation loss: 1.7373, acc: 0.6083
Epoch 4/5
----------
train loss: 1.7580, acc: 0.5732
validation loss: 1.7261, acc: 0.6139
Epoch 5/5
----------
train loss: 1.7261, acc: 0.6076
validation loss: 1.7188, acc: 0.6060
Current Best accuracy: 0.6059726254666113
Current Best parameters: (16, 0.0001, 0.0001)
Batch size: 16, Learning rate: 0.0001, Weight decay: 0.001
Epoch 1/5
----------
train loss: 1.7115, acc: 0.6168
validation loss: 1.7106, acc: 0.6012
Epoch 2/5
----------
train loss: 1.6964, acc: 0.6297
validation loss: 1.7114, acc: 0.5954
Epoch 3/5
----------
train loss: 1.6890, acc: 0.6334
validation loss: 1.6996, acc: 0.6147
Epoch 4/5
----------
train loss: 1.6822, acc: 0.6352
validation loss: 1.7

train loss: 2.1337, acc: 0.1401
validation loss: 2.0430, acc: 0.2310
Epoch 5/5
----------
train loss: 2.1301, acc: 0.1439
validation loss: 2.1085, acc: 0.1655
Batch size: 32, Learning rate: 0.1, Weight decay: 0.001
Epoch 1/5
----------
train loss: 2.1348, acc: 0.1387
validation loss: 2.1850, acc: 0.0890
Epoch 2/5
----------
train loss: 2.1448, acc: 0.1288
validation loss: 2.0430, acc: 0.2310
Epoch 3/5
----------
train loss: 2.1206, acc: 0.1530
validation loss: 2.1850, acc: 0.0890
Epoch 4/5
----------
train loss: 2.1283, acc: 0.1450
validation loss: 2.1085, acc: 0.1655
Epoch 5/5
----------
train loss: 2.1263, acc: 0.1472
validation loss: 2.1270, acc: 0.1439
Batch size: 32, Learning rate: 0.1, Weight decay: 0.01
Epoch 1/5
----------
train loss: 2.1433, acc: 0.1287
validation loss: 2.1685, acc: 0.1056
Epoch 2/5
----------
train loss: 2.1450, acc: 0.1270
validation loss: 2.1850, acc: 0.0890
Epoch 3/5
----------
train loss: 2.1423, acc: 0.1298
validation loss: 2.1850, acc: 0.0890
Epoch 4/5
